In [4]:
import os
import random
import shutil
import SimpleITK as sitk

In [5]:



# ---------------------------
# 1. Split the Data into Train/Val
# ---------------------------
# Set a seed for reproducibility
random.seed(42)



# Define the path to the directory containing the patient folders.
train_dir = '../dataset/dataset_extracted/train'  # Adjust this path if necessary

# List all folders that start with "patient" and are directories.
patient_dirs = [d for d in os.listdir(train_dir) 
                if d.startswith("patient") and os.path.isdir(os.path.join(train_dir, d))]
patient_dirs.sort()  # Ensure a consistent order before shuffling

print("Total number of patient folders:", len(patient_dirs))

# Shuffle the patient folders to randomize the split
random.shuffle(patient_dirs)

# Compute the split index (80% for training, 20% for validation)
split_index = int(0.8 * len(patient_dirs))
train_patients = patient_dirs[:split_index]
val_patients = patient_dirs[split_index:]

print("Number of training patients:", len(train_patients))
print("Number of validation patients:", len(val_patients))

# Optionally, display the lists
print("\nTraining patients:")
print(train_patients)

print("\nValidation patients:")
print(val_patients)

# Optional: Copy the split patient folders to separate directories.
# This step physically separates the training and validation data.
train_out_dir = '../dataset/dataset_extracted/train_split'
val_out_dir = '../dataset/dataset_extracted/val_split'

os.makedirs(train_out_dir, exist_ok=True)
os.makedirs(val_out_dir, exist_ok=True)

# Copy training patient folders
for patient in train_patients:
    src = os.path.join(train_dir, patient)
    dst = os.path.join(train_out_dir, patient)
    shutil.copytree(src, dst, dirs_exist_ok=True)

# Copy validation patient folders
for patient in val_patients:
    src = os.path.join(train_dir, patient)
    dst = os.path.join(val_out_dir, patient)
    shutil.copytree(src, dst, dirs_exist_ok=True)

print("\nData splitting complete.")
print(f"Training data copied to: {train_out_dir}")
print(f"Validation data copied to: {val_out_dir}")


Total number of patient folders: 98
Number of training patients: 78
Number of validation patients: 20

Training patients:
['patient088', 'patient062', 'patient065', 'patient010', 'patient034', 'patient027', 'patient002', 'patient071', 'patient016', 'patient063', 'patient066', 'patient011', 'patient054', 'patient077', 'patient053', 'patient093', 'patient051', 'patient043', 'patient068', 'patient031', 'patient058', 'patient075', 'patient047', 'patient033', 'patient100', 'patient070', 'patient095', 'patient009', 'patient039', 'patient079', 'patient083', 'patient049', 'patient046', 'patient044', 'patient099', 'patient052', 'patient086', 'patient013', 'patient037', 'patient024', 'patient041', 'patient042', 'patient019', 'patient081', 'patient073', 'patient059', 'patient008', 'patient035', 'patient082', 'patient048', 'patient003', 'patient017', 'patient040', 'patient069', 'patient023', 'patient061', 'patient025', 'patient006', 'patient007', 'patient022', 'patient050', 'patient064', 'patient0

In [6]:
# Set the target in-plane resolution (width and height) in mm.
target_resolution = [1.25, 1.25]

train_out_dir = '../dataset/dataset_extracted/train_split'
val_out_dir = '../dataset/dataset_extracted/val_split'

def resample_image(image, target_resolution, interpolator=sitk.sitkLinear):
    """
    Resample a 2D or 3D image so that its first two dimensions (x and y) have the specified resolution.
    The third dimension (if present) is left unchanged.
    """
    original_spacing = image.GetSpacing()   # e.g., (sx, sy, [sz])
    original_size = image.GetSize()           # e.g., (nx, ny, [nz])
    
    # Compute the new size for x and y
    new_size = list(original_size)
    new_size[0] = int(round(original_size[0] * original_spacing[0] / target_resolution[0]))
    new_size[1] = int(round(original_size[1] * original_spacing[1] / target_resolution[1]))
    
    # Create new spacing: update x and y; leave others unchanged.
    new_spacing = list(original_spacing)
    new_spacing[0] = target_resolution[0]
    new_spacing[1] = target_resolution[1]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetInterpolator(interpolator)
    
    return resampler.Execute(image)

# Process both train_split and val_split directories.
base_dirs = [train_out_dir, val_out_dir]

for base_dir in base_dirs:
    if not os.path.exists(base_dir):
        print("Directory does not exist:", base_dir)
        continue
    
    print("Processing directory:", base_dir)
    for patient in os.listdir(base_dir):
        patient_path = os.path.join(base_dir, patient)
        if os.path.isdir(patient_path):
            for file in os.listdir(patient_path):
                # Process only files with "frame" in their name, excluding those with "4d".
                if 'frame' in file and '4d' not in file and file.endswith('.nii'):
                    file_path = os.path.join(patient_path, file)
                    
                    # Read the image using SimpleITK.
                    image = sitk.ReadImage(file_path)
                    spacing = image.GetSpacing()
                    
                    # Check if x and y spacing are already as desired (within a small tolerance).
                    if abs(spacing[0] - target_resolution[0]) < 1e-6 and abs(spacing[1] - target_resolution[1]) < 1e-6:
                        print("Skipping already resampled:", file_path)
                        continue
                    
                    print("Resampling:", file_path)
                    
                    # Use nearest neighbor interpolation for ground truth images to preserve labels.
                    if '_gt' in file:
                        interp = sitk.sitkNearestNeighbor
                    else:
                        interp = sitk.sitkLinear
                    
                    # Resample the image.
                    resampled_image = resample_image(image, target_resolution, interpolator=interp)
                    
                    # Overwrite the original file with the resampled image.
                    sitk.WriteImage(resampled_image, file_path)

print("Resampling for train_split and val_split complete.")

Processing directory: ../dataset/dataset_extracted/train_split
Resampling: ../dataset/dataset_extracted/train_split/patient088/patient088_frame01.nii


NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame01_gt.nii has unexpected scales in sform



Resampling: ../dataset/dataset_extracted/train_split/patient088/patient088_frame01_gt.nii
Resampling: ../dataset/dataset_extracted/train_split/patient088/patient088_frame12.nii


NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame12.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame12.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame12_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/train_split/patient088/patient088_frame12_gt.nii has unexpected scales in sform



Resampling: ../dataset/dataset_extracted/train_split/patient088/patient088_frame12_gt.nii
Resampling: ../dataset/dataset_extracted/train_split/patient062/patient062_frame01.nii
Resampling: ../dataset/dataset_extracted/train_split/patient062/patient062_frame01_gt.nii


...

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient097/patient097_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient097/patient097_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient097/patient097_frame11.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient097/patient097_frame11.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient097/patient097_frame11_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient097/patient097_frame11_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient004/patient004_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../da

Resampling: ../dataset/dataset_extracted/val_split/patient097/patient097_frame01_gt.nii
Resampling: ../dataset/dataset_extracted/val_split/patient097/patient097_frame11.nii
Resampling: ../dataset/dataset_extracted/val_split/patient097/patient097_frame11_gt.nii
Resampling: ../dataset/dataset_extracted/val_split/patient004/patient004_frame01.nii
Resampling: ../dataset/dataset_extracted/val_split/patient004/patient004_frame01_gt.nii


NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient004/patient004_frame15.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient004/patient004_frame15.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient004/patient004_frame15_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient004/patient004_frame15_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient015/patient015_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient015/patient015_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient015/patient015_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../datas

Resampling: ../dataset/dataset_extracted/val_split/patient004/patient004_frame15.nii
Resampling: ../dataset/dataset_extracted/val_split/patient004/patient004_frame15_gt.nii
Resampling: ../dataset/dataset_extracted/val_split/patient015/patient015_frame01.nii
Resampling: ../dataset/dataset_extracted/val_split/patient015/patient015_frame01_gt.nii
Resampling: ../dataset/dataset_extracted/val_split/patient015/patient015_frame10.nii
Resampling: ../dataset/dataset_extracted/val_split/patient015/patient015_frame10_gt.nii
Resampling: ../dataset/dataset_extracted/val_split/patient084/patient084_frame01.nii
Resampling: ../dataset/dataset_extracted/val_split/patient084/patient084_frame01_gt.nii
Resampling: ../dataset/dataset_extracted/val_split/patient084/patient084_frame10.nii
Resampling: ../dataset/dataset_extracted/val_split/patient084/patient084_frame10_gt.nii
Resampling for train_split and val_split complete.


NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient015/patient015_frame10_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient015/patient015_frame10_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient084/patient084_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient084/patient084_frame01.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient084/patient084_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient084/patient084_frame01_gt.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../dataset/dataset_extracted/val_split/patient084/patient084_frame10.nii has unexpected scales in sform

NiftiImageIO (0x55c7d916dc30): ../da